In [279]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)


/tmp/ipykernel_36319/1266190050.py:33: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df = charger_csv(partie1, partie2, partie3, partie4)


In [280]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[2877204002, 4210335003, 5615793001, 603365301..."
1,Name,object,14,0.00,779583,"[SERVICEMASTER PROFESSIONAL SER, CAPITAL CARDS..."
2,City,object,30,0.00,32581,"[OCEANSIDE, BLOSSOM, MILLVALE, Laceyville, STR..."
3,State,object,14,0.00,51,"[GA, FL, CA, TX, WI]"
4,Zip,int64,0,0.00,33611,"[7738, 16506, 91766, 70583, 21801]"
5,Bank,object,1559,0.17,5802,"[NATIONAL BANK OF ARIZONA, WESTAMERICA BANK, B..."
6,BankState,object,1566,0.17,56,"[CA, nan, OH, TX]"
7,NAICS,int64,0,0.00,1312,"[811111, 454210, 0, 339112, 448140]"
8,ApprovalDate,object,0,0.00,9859,"[26-Dec-96, 15-Aug-97, 25-Apr-06, 18-Apr-96, 1..."
9,ApprovalFY,object,0,0.00,71,"[2008, 2005, 2007, 2005, 2002]"


In [281]:
df.shape

(899164, 27)

In [282]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: NAICS, dtype: int64

In [283]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

1.0    502830
2.0    193520
Name: NewExist, dtype: int64

In [284]:
df.shape

(696350, 27)

In [285]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [286]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
samples_bad_lol['MIS_Status'].value_counts()

CHGOFF    140687
P I F       4406
Name: MIS_Status, dtype: int64

In [287]:
status_counts = df.loc[samples_bad_lol.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

print(status_counts)

CHGOFF    145262
Name: MIS_Status, dtype: int64


In [288]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)
samples_bad['MIS_Status'].value_counts()

CHGOFF    145148
Name: MIS_Status, dtype: int64

In [289]:
samples_bad['MIS_Status'].isna().sum()

0

In [290]:
status_counts = df.loc[samples_bad.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

status_counts

CHGOFF    145148
Name: MIS_Status, dtype: int64

In [291]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

N    484187
Y    195147
Name: RevLineCr, dtype: int64

In [292]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

N    608823
Y     66203
Name: LowDoc, dtype: int64

In [293]:
df.dropna(subset=['BankState', 'Bank', 'State'], inplace=True)

In [294]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [295]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [296]:
df['FranchiseCode'].value_counts()

1    493367
0    181033
Name: FranchiseCode, dtype: int64

In [297]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [298]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [299]:
df.shape

(674400, 13)

In [300]:
df.head()

,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,SBA_Appv,Approve
0,44-45,84,4,2.0,0,0,1,0,N,Y,60000,48000,1
1,72,60,2,2.0,0,0,1,0,N,Y,40000,32000,1
2,62,180,7,1.0,0,0,1,0,N,N,287000,215250,1
5,31-33,120,19,1.0,0,0,1,0,N,N,517000,387750,1
7,81,84,1,2.0,0,0,1,0,N,Y,45000,36000,1


In [301]:
df_to_csv(df, folder_path, 'SBA_Cleaned_maybe.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_maybe.csv


In [302]:
maybe_leaking_feature = ['RetainedJob', 'CreateJob']
maybe_leaking_fea = ['SBA_Appv', 'GrAppv']
df = df.drop(maybe_leaking_feature, axis=1)

In [303]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[44-45, 72, 81]"
1,Term,int64,0,0.0,377,"[84, 12, 120, 156]"
2,NoEmp,int64,0,0.0,499,"[6, 50, 1, 4, 2]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,FranchiseCode,int64,0,0.0,2,"[1, 0]"
5,UrbanRural,int64,0,0.0,3,"[2, 1, 0]"
6,RevLineCr,object,0,0.0,2,"[Y, N]"
7,LowDoc,object,0,0.0,2,[N]
8,GrAppv,int64,0,0.0,16868,"[134000, 460000, 25000, 62500]"
9,SBA_Appv,int64,0,0.0,28200,"[5000, 92310, 30000, 127500, 45000]"


In [304]:
y = df['Approve']
X = df.drop('Approve', axis=1)

In [305]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [306]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [307]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [308]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(max_depth=25, min_samples_leaf=80, min_samples_split=3, n_estimators=70, max_samples=0.7, random_state=42))
])

In [309]:
set_config(transform_output="pandas")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.3f}')

Model Accuracy: 0.911
